General:
----------------------------------------------------------------------------------------------------------------------------
This notebook extracts data from the databases as sentences in the form of list of tokens (or list of words).

Prerequisites:
 - Download sentences.db from __FIXME__: Add public ling here

Sentence processing using SpaCy:
----------------------------------------------------------------------------------------------------------------------------
- Remove stop words
- Remove punctuation
- Mask numbers e.g.: 18 --> dd, 2018-->dddd, 34.54--> dd.dd
- Lookup word lemma and replace tokens with lemmas when they exist. 
  "Lemmatization usually refers to doing things properly with the use of a vocabulary and morphological analysis of 
   words, normally aiming to remove inflectional endings only and to return the base or dictionary form of a word,  
   which is known as the lemma." (https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html)


In [11]:
# -*- coding: utf-8 -*-
import os
import re
import errno
import random
import sqlite3
import subprocess
import pymysql.cursors
from random import shuffle
from __future__ import division

# Store unique documents
dois = set()

In [12]:
def prepare_directories(): 
    # FIXME: Add link to dowload db in this directory once it's created?
    try:
        os.mkdir("db")
    except OSError as exc:
        if exc.errno != errno.EEXIST:
            raise
        pass
    try:
        os.mkdir("models")
    except OSError as exc:
        if exc.errno != errno.EEXIST:
            raise
        pass

In [13]:
# Create directories to store database and models
prepare_directories()

In [ ]:
# Connect to db
def connect_to_db():
    database = "db/sentences.db"
    conn = create_connection(database)
    return conn

In [ ]:
# Connect to DB
def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return None

In [ ]:
def cleanup_token(tkn):
    new_tkn = tkn.strip().strip(',').strip('.')
    if len(new_tkn)>=2 and new_tkn[0] == '(' and new_tkn[len(new_tkn)-1]==')':
        new_tkn=new_tkn.rstrip(')').lstrip('(')
    return new_tkn

In [ ]:
# Get sentences
def get_sentences_from_db(conn):
    sentences = {}
    cur = conn.cursor()
    cur.execute("select sentence, haspolys, id, docid, processed_sentence from sentences") 
    rows = cur.fetchall()
 
    for row in rows:
        # Use regex to preserve spaces between parentheses
        tokens = re.split(r"\s+(?=[^()]*(?:\(|$))", row[0].rstrip("."))
        new_tokens = list(map(lambda token:cleanup_token(token),tokens))
        sentences.append(new_tokens)

    # Returns sentences as list of tokens where each sentence is broken into words using spaces except within parenthesis
    return sentences

# Same some sentences for validation

In [ ]:
# Connects to db
connection = connect_to_db()

In [ ]:
# Gets sentences and write training and testing files
sentences = get_sentences_from_db(connection)